In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage import measure

def load_image(image_path, type="grey"):
    #Load image
    img = cv2.imread(image_path)

    #Ensre image is in greyscale
    if type == "grey":
        img_ret = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    elif type == "color":
        img_ret = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    else:
        img_ret = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    return img_ret

def cluster_segment_blobs(image):
    # Label connected components (blobs)
    labels = measure.label(image, connectivity=2)

    return labels

def segmentation_methods(original_image):
    # Global Thresholding
    thresh_global = cv2.inRange(original_image, 60, 105)

    #OTSU Thresholding
    _, thresh_otsu = cv2.threshold(original_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # K-means clustering
    pixels = original_image.reshape(-1, 1)# Flatten the image to 1D array of pixels
    kmeans = KMeans(n_clusters=8, random_state=42)# Apply KMeans clustering
    kmeans.fit(pixels)
    segmented_kmeans = kmeans.labels_.reshape(original_image.shape)
    segmented_kmeans[segmented_kmeans>5] = 255

    return (thresh_global, thresh_otsu, segmented_kmeans)

def overlay_mask_on_image(image, mask, color=(0,255,0), alpha=0.4):
    #Ensure image is in colour scale so that a highlight layer can be added
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    #Create the image that will be returned
    overlay = image.copy()

    # Create color mask
    colored_mask = np.zeros_like(overlay)
    colored_mask[mask==255] = color 

    # Blend the images with some transprency
    cv2.addWeighted(colored_mask, alpha, overlay, 1 - alpha, 0, overlay)
    return overlay

def plot_segmentations_and_save(titles_top, titles_bottom, images_top, images_bottom, save_path):
    total = len(images_top)
    plt.figure(figsize=(15,8))

    # Top row: grayscale segmentations
    for i, (img, title) in enumerate(zip(images_top, titles_top)):
        plt.subplot(2, total, i+1)
        plt.imshow(img, cmap='gray')
        plt.title(title)
        plt.axis('off')

    # Bottom row: overlays on original color
    for i, (img, title) in enumerate(zip(images_bottom, titles_bottom)):
        plt.subplot(2, total, total + i + 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"Plot saved as {save_path}")

def plot_clustering(image, labels, save_path):
    # Plot original image and labeled blobs side-by-side
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Segmented Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(labels, cmap='nipy_spectral')
    plt.title('Labels')
    plt.axis('off')

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"Plot saved as {save_path}")

def plot_two_images_side_by_side(image1, image2, save_path, title1='Original Image', title2='Segmentation Aim'):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(image1, cmap='gray')
    plt.title(title1)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(image2)
    plt.title(title2)
    plt.axis('off')

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"Plot saved as {save_path}")

if __name__ == "__main__":
    #Initialise image paths
    img_path = "images/Example_Image.png"
    img_path_segment = "images/Example_Image_Segmented.png"
    img_path_segment_Mask = "images/Example_Image_Segmented_Mask.png"

    #Load images
    original_image = load_image(img_path, "grey")
    segmented_image = load_image(img_path_segment, "color")
    segmented_mask_image = load_image(img_path_segment_Mask, "grey")

    #Appply arious segmentations
    various_segmentations = segmentation_methods(original_image)

    # Create overlays of original image with the segmentation on
    overlays = [overlay_mask_on_image(original_image, mask) for mask in various_segmentations]
    
    #Create and overlay with for the expected segmentation
    transformed_segmented_mask_image = segmented_mask_image[segmented_mask_image==1] = 255
    ground_truth_seg = overlay_mask_on_image(original_image, segmented_mask_image)

    #Cluster the segmented image to show 
    segmented_mask_image[segmented_mask_image==1]=255
    clustered_labels = cluster_segment_blobs(segmented_mask_image)

    #Make the plots
    plot_two_images_side_by_side(original_image, segmented_image, "images/Aim_Of_Segmentation")
    plot_clustering(segmented_mask_image, clustered_labels, "images/Clustering_A_Segmented_Image")
    plot_segmentations_and_save(
        titles_top=["Original Image", "Global Threshold", "Otsu Threshold", "K Means Cluster"],
        titles_bottom=["Ground Truth", "Global Threshold Overlay", "Otsu Threshold Overlay", "K Means Cluster Overlay"],
        images_top=[original_image, *various_segmentations],
        images_bottom=[ground_truth_seg, *overlays],
        save_path="images/Segmentaion_Of_Example_Image")

Plot saved as images/Aim_Of_Segmentation
Plot saved as images/Clustering_A_Segmented_Image
Plot saved as images/Segmentaion_Of_Example_Image
